In [8]:
import os
import pandas as pd
import numpy as np
import urllib
import seaborn as sns
import matplotlib.pyplot as plt
import calendar
%matplotlib inline

In [9]:
path = r'C:\Users\student\Documents\Alistair CoAgMET Projects\Pythonstuff\QC\normals\anomalies\temperature_anomalies'

avgt_figpath = r'C:\Users\student\Documents\Alistair CoAgMET Projects\Pythonstuff\QC\normals\anomalies\temperature_anomalies\figures\monthly\CoAg-CoAg'
mint_figpath = r'C:\Users\student\Documents\Alistair CoAgMET Projects\Pythonstuff\QC\minimums\anomalies\temperature_anomalies\figures\monthly\CoAg-CoAg'
maxt_figpath = r'C:\Users\student\Documents\Alistair CoAgMET Projects\Pythonstuff\QC\maximums\anomalies\temperature_anomalies\figures\monthly\CoAg-CoAg'

avgt_tabpath = r'C:\Users\student\Documents\Alistair CoAgMET Projects\Pythonstuff\QC\normals\anomalies\temperature_anomalies\tables\monthly\CoAg-CoAg'
mint_tabpath = r'C:\Users\student\Documents\Alistair CoAgMET Projects\Pythonstuff\QC\minimums\anomalies\temperature_anomalies\tables\monthly\CoAg-CoAg'
maxt_tabpath = r'C:\Users\student\Documents\Alistair CoAgMET Projects\Pythonstuff\QC\maximums\anomalies\temperature_anomalies\tables\monthly\CoAg-CoAg'

os.chdir(path)

xl_file = r"C:\Users\student\Documents\Alistair CoAgMET Projects\Pythonstuff\QC\colocation_metadata.xlsx"
csv = r'C:\Users\student\Documents\Alistair CoAgMET Projects\Pythonstuff\QC\normals\anomalies\temperature_anomalies\vessel.csv'

In [10]:
xl = pd.ExcelFile(xl_file)

meta = xl.parse('metadata')
meta['sdate'] = meta.sdate.dt.date
meta['edate'] = meta.edate.dt.date
meta = meta.drop(meta.columns[3],axis=1)

In [11]:
sns.set_style('dark')
n=0
for index,rows in meta.iterrows():
    try:
        print('Grabbing CoAg data for ' + str(meta.ID[n]))
        urllib.request.urlretrieve(
            'http://coagmet.colostate.edu/cgi-bin/web_services.pl?type=daily&sids=' + str(meta.ID[n]) + '&sdate=' + str(meta.sdate[n]) + '&edate=' + str(meta.edate[n]) + '&elems=tave,tmax,tmin',
            filename=csv)
        coag = pd.read_csv(csv)
        coag = coag.reset_index()
    except Exception as e:
        print('Something went wrong while calling the CoAg data for ' + str(meta.ID[n]) + ': ' + e.args[0])
    try:
        print('Cleaning CoAg data for ' + str(meta.ID[n]))
        coag.columns.values[0] = 'station'
        coag.columns.values[1] = 'date'
        coag.columns.values[2] = 'CoAg_tmean'
        coag.columns.values[3] = 'CoAg_tmax'
        coag.columns.values[4] = 'CoAg_tmin'

        coag['date'] = pd.to_datetime(coag.date)
        coag['julian'] = coag.date.dt.strftime('%j')

        coag.CoAg_tmean = coag.CoAg_tmean.astype(float)
        coag.CoAg_tmax = coag.CoAg_tmax.astype(float)
        coag.CoAg_tmin = coag.CoAg_tmin.astype(float)

        coag.CoAg_tmean = 32 + (coag.CoAg_tmean * (9/5))
        coag.CoAg_tmax = 32 + (coag.CoAg_tmax * (9/5))
        coag.CoAg_tmin = 32 + (coag.CoAg_tmin * (9/5))

        coag=coag[:-1]
    except Exception as e:
        print('Something went wrong while cleaning the CoAg data for ' + str(meta.ID[n]) + ': ' + e.args[0])
    try:
        print('Calculating normals for ' + str(meta.ID[n]))
        normal = coag.groupby(['julian']).mean().reset_index()
    except Exception as e:
        print('Something went wrong while calculating normals for ' + str(meta.ID[n]) + ': ' + e.args[0])
    try:
        print('Merging the observed data and normals for ' + str(meta.ID[n]))
        anomaly = coag.merge(normal,'outer',left_on='julian',right_on='julian',suffixes=('_observed','_normal'))
    except Exception as e:
        print('Something went wrong while merging the observed data and normals for ' + str(meta.ID[n]) + ': ' + e.args[0])
    try:
        print('Cleaning up the merged dataset for ' + str(meta.ID[n]))
        avgt_only = ['CoAg_tmax_observed','CoAg_tmin_observed','CoAg_tmax_normal','CoAg_tmin_normal']
        maxt_only = ['CoAg_tmean_observed','CoAg_tmin_observed','CoAg_tmean_normal','CoAg_tmin_normal']
        mint_only = ['CoAg_tmean_observed','CoAg_tmax_observed','CoAg_tmean_normal','CoAg_tmax_normal']

        avgt_anomaly = anomaly.drop(avgt_only, axis=1)
        maxt_anomaly = anomaly.drop(maxt_only, axis=1)
        mint_anomaly = anomaly.drop(mint_only, axis=1)

        avgt_anomaly = avgt_anomaly[['date','julian','CoAg_tmean_observed','CoAg_tmean_normal']]
        maxt_anomaly = maxt_anomaly[['date','julian','CoAg_tmax_observed','CoAg_tmax_normal']]
        mint_anomaly = mint_anomaly[['date','julian','CoAg_tmin_observed','CoAg_tmin_normal']]

        avgt_anomaly['anomaly'] = avgt_anomaly['CoAg_tmean_observed'] - avgt_anomaly['CoAg_tmean_normal']
        maxt_anomaly['anomaly'] = maxt_anomaly['CoAg_tmax_observed'] - maxt_anomaly['CoAg_tmax_normal']
        mint_anomaly['anomaly'] = mint_anomaly['CoAg_tmin_observed'] - mint_anomaly['CoAg_tmin_normal']

        avgt_anomaly['date'] = pd.to_datetime(avgt_anomaly.date)
        maxt_anomaly['date'] = pd.to_datetime(maxt_anomaly.date)
        mint_anomaly['date'] = pd.to_datetime(mint_anomaly.date)
        
        avgt_anomaly['year'] = avgt_anomaly.date.dt.year
        maxt_anomaly['year'] = maxt_anomaly.date.dt.year
        mint_anomaly['year'] = mint_anomaly.date.dt.year
        
        avgt_anomaly['month'] = avgt_anomaly.date.dt.month
        maxt_anomaly['month'] = maxt_anomaly.date.dt.month
        mint_anomaly['month'] = mint_anomaly.date.dt.month
        
        avgt_anomaly['month'] = avgt_anomaly['month'].apply(lambda x: calendar.month_abbr[x])
        maxt_anomaly['month'] = maxt_anomaly['month'].apply(lambda x: calendar.month_abbr[x])
        mint_anomaly['month'] = mint_anomaly['month'].apply(lambda x: calendar.month_abbr[x])
    except Exception as e:
        print('Something went wrong while cleaning up the merged dataset for ' + str(meta.ID[n]) + ': ' + e.args[0])
    try:
        print('Calculating average yearly anomalies for ' + str(meta.ID[n]))
        ya_avgt_anomaly = avgt_anomaly.groupby(['month']).mean().reset_index()
        ya_maxt_anomaly = maxt_anomaly.groupby(['month']).mean().reset_index()
        ya_mint_anomaly = mint_anomaly.groupby(['month']).mean().reset_index()
    except Exception as e:
        print('Something went wrong while calculating average yearly anomalies for ' + str(meta.ID[n]) + ': ' + e.args[0])
    try:
        print('Storing average yearly anomaly tables for ' + str(meta.ID[n]))
        ya_avgt_anomaly.to_csv(avgt_tabpath + r'\CoAg-CoAg_' + str(meta.ID[n]) + '_avgt_monthly_anomaly.csv')
        ya_maxt_anomaly.to_csv(maxt_tabpath + r'\CoAg-CoAg_' + str(meta.ID[n]) + '_maxt_monthly_anomaly.csv')
        ya_mint_anomaly.to_csv(mint_tabpath + r'\CoAg-CoAg_' + str(meta.ID[n]) + '_mint_monthly_anomaly.csv')
    except Exception as e:
        print('Something went wrong while storing the average yearly anomaly tables for ' + str(meta.ID[n]) + ': ' + e.args[0])
    try:
        print('Visualizing and storing average yearly anomaly figures for ' + str(meta.ID[n]))
        avgt_ax = sns.catplot(x='month',y='anomaly',data=ya_avgt_anomaly,kind='bar',palette='colorblind',height=8.27,aspect=11.7/8.27)
        plt.title(str(meta.ID[n]) + ' - Monthly Average Daily Temperature Anomaly')
        plt.ylabel('Monthly Deviation from Daily Average Temperature Normals (°F)')
        plt.xlabel('Month')
        plt.savefig(avgt_figpath + r'\CoAg-CoAg_' + str(meta.ID[n]) + '_avgt_monthly_anomaly.png',bbox_inches='tight')
        plt.clf()
        maxt_ax = sns.catplot(x='month',y='anomaly', data=ya_maxt_anomaly,kind='bar',palette='colorblind',height=8.27,aspect=11.7/8.27)
        plt.title(str(meta.ID[n]) + ' - Monthly Average Maximum Temperature Anomaly')
        plt.ylabel('Monthly Deviation from Daily Maximum Temperature Normals (°F)')
        plt.xlabel('Month')
        plt.savefig(maxt_figpath + r'\CoAg-CoAg_' + str(meta.ID[n]) + '_maxt_monthly_anomaly.png', bbox_inches='tight')
        plt.clf()
        mint_ax = sns.catplot(x='month',y='anomaly',data=ya_mint_anomaly,kind='bar',palette='colorblind',height=8.27,aspect=11.7/8.27)
        plt.title(str(meta.ID[n]) + ' - Monthly Average Minimum Temperature Anomaly')
        plt.ylabel('Monthly Deviation from Daily Minimum Temperature Normals (°F)')
        plt.xlabel('Month')
        plt.savefig(mint_figpath + r'\CoAg-CoAg_' + str(meta.ID[n]) + '_mint_monthly_anomaly.png',bbox_inches='tight')
        plt.close()
    except Exception as e:
        print('Something went wrong while visualizing and storing the anomaly data for ' + str(meta.ID[n]) + ': ' + e.args[0])
    n=n+1

Grabbing CoAg data for AKR02
Cleaning CoAg data for AKR02
Calculating normals for AKR02
Merging the observed data and normals for AKR02
Cleaning up the merged dataset for AKR02
Calculating average yearly anomalies for AKR02
Storing average yearly anomaly tables for AKR02
Visualizing and storing average yearly anomaly figures for AKR02
Grabbing CoAg data for ALT01
Cleaning CoAg data for ALT01
Calculating normals for ALT01
Merging the observed data and normals for ALT01
Cleaning up the merged dataset for ALT01
Calculating average yearly anomalies for ALT01
Storing average yearly anomaly tables for ALT01
Visualizing and storing average yearly anomaly figures for ALT01
Grabbing CoAg data for AVN01
Cleaning CoAg data for AVN01
Calculating normals for AVN01
Merging the observed data and normals for AVN01
Cleaning up the merged dataset for AVN01
Calculating average yearly anomalies for AVN01
Storing average yearly anomaly tables for AVN01
Visualizing and storing average yearly anomaly figures

Grabbing CoAg data for GUN01
Cleaning CoAg data for GUN01
Calculating normals for GUN01
Merging the observed data and normals for GUN01
Cleaning up the merged dataset for GUN01
Calculating average yearly anomalies for GUN01
Storing average yearly anomaly tables for GUN01
Visualizing and storing average yearly anomaly figures for GUN01
Grabbing CoAg data for HEB01
Cleaning CoAg data for HEB01
Calculating normals for HEB01
Merging the observed data and normals for HEB01
Cleaning up the merged dataset for HEB01
Calculating average yearly anomalies for HEB01
Storing average yearly anomaly tables for HEB01
Visualizing and storing average yearly anomaly figures for HEB01
Grabbing CoAg data for HLY01
Cleaning CoAg data for HLY01
Calculating normals for HLY01
Merging the observed data and normals for HLY01
Cleaning up the merged dataset for HLY01
Calculating average yearly anomalies for HLY01
Storing average yearly anomaly tables for HLY01
Visualizing and storing average yearly anomaly figures

Grabbing CoAg data for MTR01
Cleaning CoAg data for MTR01
Calculating normals for MTR01
Merging the observed data and normals for MTR01
Cleaning up the merged dataset for MTR01
Calculating average yearly anomalies for MTR01
Storing average yearly anomaly tables for MTR01
Visualizing and storing average yearly anomaly figures for MTR01
Grabbing CoAg data for ORM01
Cleaning CoAg data for ORM01
Calculating normals for ORM01
Merging the observed data and normals for ORM01
Cleaning up the merged dataset for ORM01
Calculating average yearly anomalies for ORM01
Storing average yearly anomaly tables for ORM01
Visualizing and storing average yearly anomaly figures for ORM01
Grabbing CoAg data for OTH01
Cleaning CoAg data for OTH01
Calculating normals for OTH01
Merging the observed data and normals for OTH01
Cleaning up the merged dataset for OTH01
Calculating average yearly anomalies for OTH01
Storing average yearly anomaly tables for OTH01
Visualizing and storing average yearly anomaly figures

<Figure size 842.4x595.44 with 0 Axes>

<Figure size 842.4x595.44 with 0 Axes>

...

<Figure size 842.4x595.44 with 0 Axes>

<Figure size 842.4x595.44 with 0 Axes>

<Figure size 842.4x595.44 with 0 Axes>

<Figure size 842.4x595.44 with 0 Axes>

<Figure size 842.4x595.44 with 0 Axes>